In [73]:
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Layout, Label, Text, GridBox, GridspecLayout, Dropdown, Tab, Button
import queries
import plotly.express as px
from pycelonis import get_celonis
from DataModel import DataModelInfo
import plotly.graph_objects as go
import math
from preprocessing import Attribute, AttributeDataType, Preprocessor
from pycelonis.celonis_api.pql.pql import PQL, PQLColumn, PQLFilter
import functools
import logging
import sys
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from pandas.api.types import is_numeric_dtype

In [ ]:
logging.disable(sys.maxsize)
login = {"celonis_url": "academic-michael-schulten-rwth-aachen-de.eu-2.celonis.cloud",
    "api_token": "ODJkZDhjNmQtYTQ2Ny00NWRlLWJkMGYtZWJjY2FjOGVhYmQyOllBYlBkRXNHV2psZ1o1MDJacmlsRVU3KytxaDdLVHY5N1lBOHJQdTJnOXR0",
    # The following 2 lines are only necessary when connecting to CPM4.5, not for IBC:
    # "api_id": "paste_here_your_api_id",
    # "username": "paste_here_your_username",
}
celonis = get_celonis(**login)
dm = celonis.datamodels.find("P2P_Wils_Course")
dm_info = DataModelInfo(dm)
p = Preprocessor(dm)
df_total_time = p.run_total_time_PQL(20, time_aggregation="DAYS")

In [103]:
df_total_time

,caseid,Case start time,Case end time,Start activity = Create Purchase Order Item,Start activity = Create Purchase Requisition Item,Start activity = Record Invoice Receipt,End activity = Block Purchase Order Item,End activity = Clear Invoice,End activity = Delete Purchase Order Item,End activity = Refuse Purchase Order,...,P2P-Activities_csv_USER_NAME_Rona Hennis,P2P-Activities_csv_USER_NAME_Roy Ortego,P2P-Activities_csv_USER_NAME_Suellen Noecker,P2P-Activities_csv_USER_NAME_Summer Fusaro,P2P-Activities_csv_USER_NAME_Tasia Snelgrove,P2P-Activities_csv_USER_NAME_Toby Narvaez,P2P-Activities_csv_USER_NAME_Yanira Gorrfried,P2P-Activities_csv_USER TYPE_A,P2P-Activities_csv_USER TYPE_B,case duration
0,1000467,2016-03-30 14:31:34,2016-04-28 14:31:34,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,5,29.0
1,1000468,2016-03-30 14:31:34,2016-04-27 14:31:34,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,3,3,28.0
2,1000469,2016-03-31 14:31:34,2016-05-07 14:31:34,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,4,2,37.0
3,100047,2016-10-06 12:51:04,2016-10-31 12:51:04,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,3,2,25.0
4,1000470,2016-03-30 14:34:02,2016-05-07 14:34:02,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,3,3,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,1000462,2016-03-30 14:31:34,2016-04-27 14:31:34,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,6,28.0
808,1000463,2016-04-01 14:31:34,2016-04-25 14:31:34,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,2,4,24.0
809,1000464,2016-04-01 14:31:34,2016-04-28 14:31:34,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,3,3,27.0
810,1000465,2016-03-31 14:31:34,2016-04-27 14:31:34,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,3,3,27.0


In [40]:
df_total_time[p.label.df_attribute_name].value_counts()

29     60
27     54
31     52
32     50
33     44
30     44
28     42
34     38
25     36
26     34
35     33
36     32
37     29
38     25
23     25
24     25
22     20
40     17
11     15
42     14
21     13
39     11
20     10
41     10
3       9
13      8
12      7
43      6
14      5
4       5
44      4
45      4
10      3
19      3
9       2
5       2
17      2
6       2
46      2
2       2
18      2
64      2
48      1
47      1
191     1
49      1
55      1
53      1
16      1
66      1
50      1
Name: case duration, dtype: int64

In [42]:
df_total_time[p.label.df_attribute_name] = df_total_time[p.label.df_attribute_name].astype(float)

In [43]:
fig = px.ecdf(df_total_time, x=p.label.df_attribute_name)


fig.update_layout({'xaxis_title': "Total trace time (days)", 'yaxis_title': "cumulative probability"})
fig.update_xaxes(range=[0, 60])
fig.show()

### Clean dataframe
This should be already done before in preprocessing. I.e. have a dataframe that only contains the attributes that shall be used for analysis and decision rule mining.
So, columns like Case start time and Case end time must be stored in a separate dataframe.
Actually not needed

In [25]:
df_cleaned = df_total_time[list(p.attributes_dict.keys()) + [p.label.df_attribute_name]]


In [76]:
df_cleaned

,Start activity = Create Purchase Order Item,Start activity = Create Purchase Requisition Item,Start activity = Record Invoice Receipt,End activity = Block Purchase Order Item,End activity = Clear Invoice,End activity = Delete Purchase Order Item,End activity = Refuse Purchase Order,P2P-Activities_csv_ACTIVITY_EN_Block Purchase Order Item (occurence),P2P-Activities_csv_ACTIVITY_EN_Change Currency (occurence),P2P-Activities_csv_ACTIVITY_EN_Change Price (occurence),...,P2P-Activities_csv_USER_NAME_Rona Hennis,P2P-Activities_csv_USER_NAME_Roy Ortego,P2P-Activities_csv_USER_NAME_Suellen Noecker,P2P-Activities_csv_USER_NAME_Summer Fusaro,P2P-Activities_csv_USER_NAME_Tasia Snelgrove,P2P-Activities_csv_USER_NAME_Toby Narvaez,P2P-Activities_csv_USER_NAME_Yanira Gorrfried,P2P-Activities_csv_USER TYPE_A,P2P-Activities_csv_USER TYPE_B,long_time
0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,5,Hallo
1,0,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,3,3,Hallo
2,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,4,2,Hallo
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,3,2,Hallo
4,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,3,3,Hallo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,0
808,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2,4,0
809,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,3,3,0
810,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,3,3,0


In [75]:
df_cleaned.loc[:300, 'long_time'] = "Hallo"

In [45]:
long_time = 40
df_cleaned['long_time'] = np.where(df_cleaned[p.label.df_attribute_name] >= long_time, 1, 0)

In [48]:
df_cleaned = df_cleaned.drop(p.label.df_attribute_name, axis=1)

In [19]:
import wittgenstein as lw

In [20]:
ripper_clf = lw.RIPPER()

In [77]:
ripper_clf.fit(df_cleaned, max_rule_conds=2, n_discretize_bins=3, class_feat="long_time", pos_class="Hallo", k=10, random_state=None, max_rules=3)

irep/ripper.py: .fit: DeprecationWarning:


In the future, you should assign these parameters when initializating classifier instead of during model fitting: ['max_rule_conds', 'n_discretize_bins', 'k', 'random_state', 'max_rules']


/home/michaelschulten/miniconda3/envs/one_click_analysis/lib/python3.8/site-packages/wittgenstein/base.py:127: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [70]:
ripper_clf.ruleset_.rules[0].conds[0].feature

'P2P-Activities_csv_ACTIVITY_EN_Change Price (occurence)'

In [85]:
pred = ripper_clf.predict(df_cleaned.drop('long_time', axis=1))
avg_True = df_total_time[np.array(pred)]['case duration'].mean()
avg_False = df_total_time[~np.array(pred)]['case duration'].mean()

/home/michaelschulten/miniconda3/envs/one_click_analysis/lib/python3.8/site-packages/wittgenstein/base.py:127: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [87]:
avg_False

30.890522875816995

In [52]:
ripper_clf.predict_proba(df_cleaned)

/home/michaelschulten/miniconda3/envs/one_click_analysis/lib/python3.8/site-packages/wittgenstein/base.py:127: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/home/michaelschulten/miniconda3/envs/one_click_analysis/lib/python3.8/site-packages/wittgenstein/base.py:127: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



array([[0.97783934, 0.02216066],
       [0.97783934, 0.02216066],
       [0.97783934, 0.02216066],
       ...,
       [0.97783934, 0.02216066],
       [0.97783934, 0.02216066],
       [0.97783934, 0.02216066]])

In [119]:
class DecisionRuleMiner:
    def __init__(self, df, class_label, attrbute_labels, pos_class: str=None, threshold:float = None):
        self.df = df
        self.attrbute_labels = attrbute_labels
        self.preprocessed_label_col = "Decisionrulelabel"
        self.class_label = class_label
        self.attribute_label = attrbute_labels
        self.max_rule_conds = 2  # probably shouldn't change this to a higher value since it will become too difficult to read
        self.n_discretize_bins = 2  # this should be fixed. Maybe can also use 3
        self.pos_class = pos_class  # the value of the positive class. Leave at None if class is numerical
        self.k = 2  # number of iterations
        self.random_state=42
        self.max_rules = 3  # this variable can be changed to make rules simpler or more elaborate
        self.threshold = threshold  # threshold to define positive class for numerical class. Positive class is >= threshold
        self.label_df = self.get_label_df()

    def get_label_df(self):
        """

        :param df:
        :return:
        """
        label_df = pd.DataFrame()
        # create new label column if original is numerical
        if is_numeric_dtype(self.df[self.class_label]) and self.threshold is not None:

            label_df[self.preprocessed_label_col] = np.where(self.df[self.class_label] >= self.threshold, 1, 0)
            self.pos_class = 1

        else:
            label_df[self.preprocessed_label_col] = self.df[self.class_label]
        return label_df

    def run_pipeline(self):
        """ Run the pipeline and return all the interesting metrics

        """
        clf = self._fit()
        pred = self._get_prediction_labels(clf)
        true_n, false_p, false_n, true_p, recall_p, recall_n, precision_p, precision_n = self._get_confusion_matrix(pred)
        avg_True, avg_False = self._get_avg_label_value(pred)
        ret_dict = {
            "true_n": true_n,
            "false_p": false_p,
            "false_n": false_n,
            "true_p": true_p,
            "recall_p": recall_p,
            "recall_n": recall_n,
            "precision_p": precision_p,
            "precision_n": precision_n,
            "avg_True": avg_True,
            "avg_False": avg_False,
        }
        return ret_dict

    def _fit(self):
        ripper_clf = lw.RIPPER(max_rule_conds=self.max_rule_conds, n_discretize_bins=self.n_discretize_bins,  k=self.k, random_state=self.random_state, max_rules=self.max_rules)
        ripper_clf.fit(pd.concat([self.df[self.attrbute_labels], self.label_df]), class_feat=self.preprocessed_label_col, pos_class=self.pos_class)
        return ripper_clf

    def _get_prediction_labels(self, clf):
        pred = clf.predict(self.df[self.attrbute_labels])
        return np.array(pred)

    def _get_confusion_matrix(self, pred):
        true_labels = self.label_df[self.preprocessed_label_col]
        true_n, false_p, false_n, true_p = confusion_matrix(true_labels, pred).ravel()
        recall_p = (true_p/(true_p + false_n))
        recall_n = (true_n/(true_n + false_p))
        precision_p = (true_p/(true_p + false_p))
        precision_n = (true_n/(true_n + false_n))
        return true_n, false_p, false_n, true_p, recall_p, recall_n, precision_p, precision_n

    def _get_avg_label_value(self, pred):
        avg_True = self.df[pred][self.class_label].mean()
        avg_False = self.df[~pred][self.class_label].mean()
        return avg_True, avg_False


In [117]:
dr_miner = DecisionRuleMiner(df_total_time, p.label.df_attribute_name, p.attributes_dict.keys(), pos_class=None, threshold = 40)

In [118]:
res = dr_miner.run_pipeline()

In [111]:
res

{'true_n': 741,
 'false_p': 4,
 'false_n': 67,
 'true_p': 0,
 'recall_p': 0.0,
 'recall_n': 0.9946308724832215,
 'precision_p': 0.0,
 'precision_n': 0.9170792079207921,
 'avg_True': 35.5,
 'avg_False': 29.57549504950495}

In [ ]:
"false_p": false_p,

In [88]:
df_test = pd.DataFrame()
df_test['a'] = [1,2,41,213,12,321,421,41,421,4124]

In [90]:
df_test['a'].mean()

559.7